In [1]:
from table.experiments import load_environments
from pytorch_pretrained_bert import BertTokenizer
tknzer = BertTokenizer.from_pretrained('bert-large-uncased')

In [2]:
from pathlib import Path
base_dir = Path('/workspace/hsiehcc/pytorch_neural_symbolic_machines/')
data_dir = base_dir/'data/wikitable/wtq_preprocess_0805_no_anonymize_ent'
table_file = data_dir / 'tables.jsonl'
test_file = data_dir / 'data_split_1' / 'dev_split.jsonl'

In [3]:
test_envs = load_environments(
    [test_file],
    table_file=table_file,
    table_representation_method='canonical',
    bert_tokenizer=tknzer
)

2831 examples in dataset.
2108 tables.
creating environment #0
creating environment #100
creating environment #200
creating environment #300
creating environment #400
creating environment #500
creating environment #600
creating environment #700
creating environment #800
creating environment #900
creating environment #1000
creating environment #1100
creating environment #1200
creating environment #1300
creating environment #1400
creating environment #1500
creating environment #1600
creating environment #1700
creating environment #1800
creating environment #1900
creating environment #2000
creating environment #2100
creating environment #2200
creating environment #2300
creating environment #2400
creating environment #2500
creating environment #2600
creating environment #2700
creating environment #2800
2831 environments in total


In [14]:
for env in test_envs:
    env.use_cache = False
    env.punish_extra_work = False
test_dics = {test_env.name:test_env for test_env in test_envs}

In [44]:
test=test_envs[0]

In [5]:
from nsm.parser_module.agent import PGAgent

In [7]:
model_path=base_dir/'runs/demo_run/model.best.bin'
agent = PGAgent.load(model_path, gpu_id=0).eval()

Loading table BERT model /workspace/hsiehcc/TaBERT/tabert_large_k3/model.bin
100%|██████████| 1248501532/1248501532 [01:35<00:00, 13008919.65B/s]
Table Bert Config
{
  "base_model_name": "bert-large-uncased",
  "column_delimiter": "[SEP]",
  "context_first": true,
  "column_representation": "mean_pool",
  "max_cell_len": 5,
  "max_sequence_len": 512,
  "max_context_len": 128,
  "do_lower_case": true,
  "cell_input_template": [
    "column",
    "|",
    "type",
    "|",
    "value"
  ],
  "masked_context_prob": 0.15,
  "masked_column_prob": 0.2,
  "max_predictions_per_seq": 200,
  "context_sample_strategy": "concate_and_enumerate",
  "table_mask_strategy": "column",
  "vocab_size": 30522,
  "hidden_size": 1024,
  "num_hidden_layers": 24,
  "num_attention_heads": 16,
  "hidden_act": "gelu",
  "intermediate_size": 4096,
  "hidden_dropout_prob": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "max_position_embeddings": 512,
  "type_vocab_size": 2,
  "initializer_range": 0.02,
  "layer_norm_

In [22]:
decode_results = agent.decode_examples([test], beam_size=10)

0it [00:00, ?it/s]******************************
[{'value': <bound method SimpleKGExecutor.argmax of <nsm.execution.worlds.wikitablequestions.WikiTableExecutor object at 0x7f89cf915a58>>, 'type': 'primitive_function', 'autocomplete': <bound method SimpleKGExecutor.autocomplete_argm of <nsm.execution.worlds.wikitablequestions.WikiTableExecutor object at 0x7f89cf915a58>>, 'return_type': 'entity_list', 'args': [{'types': ['entity_list']}, {'types': ['ordered_property']}]}]
['all_rows']
['all_rows']
******************************
******************************
[{'value': <bound method SimpleKGExecutor.argmin of <nsm.execution.worlds.wikitablequestions.WikiTableExecutor object at 0x7f89cf915a58>>, 'type': 'primitive_function', 'autocomplete': <bound method SimpleKGExecutor.autocomplete_argm of <nsm.execution.worlds.wikitablequestions.WikiTableExecutor object at 0x7f89cf915a58>>, 'return_type': 'entity_list', 'args': [{'types': ['entity_list']}, {'types': ['ordered_property']}]}]
['all_rows'

In [19]:
from nsm.evaluator import Evaluation

In [23]:
Evaluation.evaluate_decode_results([test],decode_results)

{'accuracy': 0.0, 'oracle_accuracy': 1.0}

In [65]:
sample = decode_results[0][0]

In [62]:
env = test.clone()

In [63]:
res = []
for tok in sample.trajectory.program:
    res.append(env.interpreter.read_token(tok))

v8 {'value': ['row_6'], 'type': 'entity_list', 'is_constant': False}
v9 {'value': ['row_5'], 'type': 'entity_list', 'is_constant': False}
v10 {'value': ['confey'], 'type': 'string_list', 'is_constant': False}


In [64]:
env.interpreter.result

['confey']

In [67]:
sample.trajectory.answer

['confey']

In [68]:
#TODO load environment step by step to modify table
#less important: copy code to script and pdb to investigate decoding workflow